In [ ]:
!pip install pyyaml==5.1

     |████████████████████████████████| 276kB 33.4MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=983a964cc403f32c9779d0c79e4257e7b7eb45eb0637c7c0419f57fe088d4224
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2MB 538kB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=25aac0096be44538f1db14d2dfad218a0e923e5aea48fadb60f2d7e3578caa63
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
Successfully built fvcore


In [ ]:
!pip install webvtt-py

In [ ]:
import torch, torchvision

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)
predictor = DefaultPredictor(cfg)

model_final_f10217.pkl: 178MB [00:08, 20.2MB/s]                           


In [ ]:
def get_predictions(image, height_thresh=0.75, person_label=0):
    outputs = predictor(image)
    im_h, im_w, _ = image.shape
    pred_classes = outputs["instances"].pred_classes.cpu().numpy()
    pred_boxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()
    signer_cnt = 1
    signer_bboxes = []
    for class_, bbox_ in zip(pred_classes, pred_boxes):
        if class_ == person_label:
            x1, y1, w, h = bbox_[0], bbox_[1], bbox_[2] - bbox_[0], bbox_[3] - bbox_[1]
            print("h=", h, "im_h=", im_h)
            if h >= im_h * height_thresh:
                signer_bboxes.append(bbox_)
                img_with_bbox = cv2.rectangle(
                    image, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2
                )
                cv2.putText(
                    img_with_bbox,
                    str(signer_cnt),
                    (int(x1) + 15, int(y1) + 35),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.9,
                    (36, 255, 12),
                    3,
                )
                signer_cnt += 1

    return signer_bboxes, img_with_bbox

In [ ]:
def get_bboxes_for_crop(image, bboxes, pad_scale=0.13):
    im_h, im_w, _ = image.shape
    # To (x1,y1, x2,y2)
    bboxes_formatted = [
        bboxes[0][0],
        bboxes[0][1],
        (bboxes[0][0] + bboxes[0][2]),
        (bboxes[0][1] + bboxes[0][3]),
    ]
    new_x1 = (
        (bboxes_formatted[0] - (pad_scale * im_w))
        if (bboxes_formatted[0] - (pad_scale * im_w)) > 0
        else 0
    )
    new_x2 = (
        (bboxes_formatted[2] + (pad_scale * im_w))
        if (bboxes_formatted[2] + (pad_scale * im_w)) < im_w
        else im_w
    )
    padded_bboxes = [new_x1, 0, new_x2, im_h]
    # To (x1,y1,w,h)
    bboxes_formatted = [
        padded_bboxes[0],
        padded_bboxes[1],
        (padded_bboxes[2] - padded_bboxes[0]),
        (padded_bboxes[3] - padded_bboxes[1]),
    ]

    return bboxes_formatted

In [ ]:
image = cv2.imread(
    "/content/drive/MyDrive/filename00500.png"
)
bboxes, image_with_txt = get_predictions(image)
new_bboxes = get_bboxes_for_crop(image, bboxes)

h= 558.51245 im_h= 720


In [ ]:
new_bboxes

[512.0556884765625, 0, 767.9443115234375, 720]

In [ ]:
!ffmpeg -i /content/drive/MyDrive/'Clear Clear Clear Body.mp4' -filter:v "crop=450:720:(512.0556884765625+767.9443115234375)/2+75:0" final.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
from webvtt import WebVTT
from tqdm import tqdm
import datetime
import copy
import hashlib
import os
import io
from joblib import Parallel, delayed
import multiprocessing


PATH = "/content/drive/MyDrive/Test"
SUBS_PATH = "/content/drive/MyDrive/Test"
SAVE_PATH = "/content/drive/MyDrive/Test"

n_cores = multiprocessing.cpu_count()

def get_hash(content):
    return hashlib.sha224(content.encode('utf-8')).hexdigest()

def load_all_subtitles(subtitle_file):
    subs = WebVTT().read(subtitle_file).captions  
    res = []
    for s_idx, s in enumerate(subs):
        start_ts = parse_ts(s.start).time()
        end_ts = parse_ts(s.end).time()
        phrase = s.text
        phrase = phrase.replace('\n', ' ')
        delta = timedelta_dt(start_ts, end_ts)
        res.append(
            {"ts_start": start_ts, 
             "ts_end": end_ts,
             "text": phrase,
             "sub_file": subtitle_file,
             "duration": delta.total_seconds(),
             "idx": s_idx}
        )
    return res


def parse_ts(ts_string):
    if ' ' in ts_string:
        ts_string = ts_string.split()[0]
    ts = datetime.datetime.strptime(ts_string, '%H:%M:%S.%f')
    return ts

def timedelta_dt(t1, t2):
    dt1 = datetime.timedelta(hours=t1.hour, minutes=t1.minute,
                             seconds=t1.second, microseconds=t1.microsecond)
    dt2 = datetime.timedelta(hours=t2.hour, minutes=t2.minute,
                             seconds=t2.second, microseconds=t2.microsecond)
    return dt2 - dt1


def merge_subtitles_delimiter_based(subs, delimiters=['.','!','?']):
  res = []
  last_delimiter_occur = -1
  start_end_indices = []
  for s_idx in range(len(subs)):
    s = subs[s_idx]
    if s["text"][-1] in delimiters or s_idx == len(subs)-1:
      start_end_indices.append((last_delimiter_occur, s_idx))
      last_delimiter_occur = s_idx

  for index, (start,end) in enumerate(start_end_indices):
    new_sub = copy.deepcopy(subs[start+1])
    new_sub["idx"] = index
    for i in subs[start+2:end+1]:
      new_sub["ts_end"] = i["ts_end"]
      new_sub["text"]+=i["text"]
      new_delta = timedelta_dt(new_sub["ts_start"], new_sub["ts_end"])
      new_sub["duration"] = new_delta.total_seconds()
    res.append(new_sub)

  return res

def crop_and_save_file(video_file, sub, mp4_save_dir, sub_save_dir):
    hash = get_hash(
        sub_save_dir + sub["text"] + str(sub["ts_start"]))
    timing_start = sub["ts_start"]
    timing_end = sub["ts_end"]
    text = sub["text"]
    
    start_h, start_m, start_s, start_msec = timing_start.hour, timing_start.minute, \
                                                timing_start.second, timing_start.microsecond // 1000
    end_h, end_m, end_s, end_msec = timing_end.hour, timing_end.minute, \
                                        timing_end.second, timing_end.microsecond // 1000

    target_vid_file = os.path.join(mp4_save_dir, hash + ".mp4")
    target_sub_file = os.path.join(sub_save_dir, hash + ".txt")
    
    try:    
        res = os.system("ffmpeg -hide_banner -loglevel warning -i {0} -ss {1}:{2}:{3}.{4}  -to {5}:{6}:{7}.{8} {9}".format(video_file, start_h, start_m, start_s, start_msec, end_h, end_m, end_s, end_msec, target_vid_file ))
        print(res)
        if res !=0:
            raise Exception('ffmpeg error')
        with io.open(target_sub_file, "w", encoding='utf-8') as f:
            f.write(text)
    except Exception as e:
        print(e, "-->", video_file)
        
    return
     
def run_for_file(file, PATH):   
    if file.endswith(".mp4"):
        video_file = os.path.join(PATH, file)
        try:
            subtitle_file = os.path.join(SUBS_PATH, file.replace("mp4", "en-GB.vtt"))
            
            if not os.path.exists(subtitle_file):
                raise Exception("Subtitle file or Info files do not exist.")

            subs = load_all_subtitles(subtitle_file)
            res_subs =  merge_subtitles_delimiter_based(subs)
        
            mp4_save_dir = os.path.join(SAVE_PATH, os.path.splitext(os.path.basename(video_file))[0], "vid")
            sub_save_dir = os.path.join(SAVE_PATH, os.path.splitext(os.path.basename(video_file))[0] ,"subs")
            os.makedirs(mp4_save_dir, exist_ok=True)
            os.makedirs(sub_save_dir, exist_ok=True)    
            i=0
            for sub in res_subs:
                i+=1
                crop_and_save_file(video_file, sub, mp4_save_dir, sub_save_dir)
            
        except Exception as e:
            print(e)
            #pass
   
Parallel(n_jobs=n_cores, backend="multiprocessing")(delayed(run_for_file)(file, PATH)
                            for file in tqdm(os.listdir(PATH)))

100%|██████████| 3/3 [00:00<00:00, 25.13it/s]


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


[None, None, None]